In [3]:
!export GROQ_API_KEY="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"


In [4]:
from crewai import Agent, Task, Crew, LLM

# Initialize Large Language Model (LLM) of your choice (see all models on our Models page)
llm = LLM(model="groq/llama-3.1-70b-versatile")

# Create your CrewAI agents with role, main goal/objective, and backstory/personality
summarizer = Agent(
    role='Documentation Summarizer', # Agent's job title/function
    goal='Create concise summaries of technical documentation', # Agent's main objective
    backstory='Technical writer who excels at simplifying complex concepts', # Agent's background/expertise
    llm=llm, # LLM that powers your agent
    verbose=True # Show agent's thought process as it completes its task
)

translator = Agent(
    role='Technical Translator',
    goal='Translate technical documentation to other languages',
    backstory='Technical translator specializing in software documentation',
    llm=llm,
    verbose=True
)

# Define your agents' tasks
summary_task = Task(
    description='Summarize this React hook documentation:\n\nuseFetch(url) is a custom hook for making HTTP requests. It returns { data, loading, error } and automatically handles loading states.',
    expected_output="A clear, concise summary of the hook's functionality",
    agent=summarizer # Agent assigned to task
)

translation_task = Task(
    description='Translate the summary to Turkish',
    expected_output="Turkish translation of the hook documentation",
    agent=translator,
    dependencies=[summary_task] # Must run after the summary task
)

# Create crew to manage agents and task workflow
crew = Crew(
    agents=[summarizer, translator], # Agents to include in your crew
    tasks=[summary_task, translation_task], # Tasks in execution order
    verbose=True
)

result = crew.kickoff()
print(result)


LiteLLM is not available, falling back to LiteLLM


ImportError: Fallback to LiteLLM is not available

In [4]:
import requests

paper_id = "2511.20636"
url = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}/related-papers?fields=title,abstract,url"

response = requests.get(url).json()

for p in response.get("data", []):
    print(p["score"], p["paper"]["title"])


In [5]:
response

{'error': 'Paper with id 2511.20636/related-papers not found'}

In [ ]:
import arxiv
from sentence_transformers import SentenceTransformer, util

# 1. Load an embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Get abstract of a given paper
paper_id = "2511.20636"  # example
search = arxiv.Search(id_list=[paper_id])
paper = list(search.results())[0]
target_abstract = paper.summary

# 3. Encode the target paper
target_emb = model.encode(target_abstract, convert_to_tensor=True)

# 4. Search for candidate papers in the same category
search_query = "cat:cs.CL"  # choose your category
results = arxiv.Search(query=search_query, max_results=200)

candidate_papers = []
candidate_abstracts = []

for r in results.results():
    if r.entry_id.endswith(paper_id):
        continue
    candidate_papers.append(r)
    candidate_abstracts.append(r.summary)

# 5. Encode candidate abstracts
candidate_embs = model.encode(candidate_abstracts, convert_to_tensor=True)

# 6. Compute similarities
cos_scores = util.cos_sim(target_emb, candidate_embs)[0]

# 7. Get top-k similar papers
top_k = 10
top_results = cos_scores.topk(top_k)

print("Top similar papers:")
for score, idx in zip(top_results.values, top_results.indices):
    p = candidate_papers[idx]
    print(f"{score:.4f} - {p.title} ({p.entry_id})")


/home/smayan/Desktop/AI-ML-DS/AI-and-ML-Course/.conda/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
2025-11-26 16:41:09.769100: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-26 16:41:09.899044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764155469.945200   12882 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDN

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_12882/2036739176.py:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  paper = list(search.results())[0]
/tmp/ipykernel_12882/2036739176.py:23: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for r in results.results():


Top similar papers:
0.4052 - ShapeGlot: Learning Language for Shape Differentiation (http://arxiv.org/abs/1905.02925v1)
0.3416 - Intelligent requirements engineering from natural language and their chaining toward CAD models (http://arxiv.org/abs/2007.07825v1)
0.3105 - Listening while Speaking and Visualizing: Improving ASR through Multimodal Chain (http://arxiv.org/abs/1906.00579v3)
0.2871 - Hierarchical Decision Making by Generating and Following Natural Language Instructions (http://arxiv.org/abs/1906.00744v5)
0.2846 - A Study on Effects of Implicit and Explicit Language Model Information for DBLSTM-CTC Based Handwriting Recognition (http://arxiv.org/abs/2008.01532v1)
0.2744 - The Go Transformer: Natural Language Modeling for Game Play (http://arxiv.org/abs/2007.03500v3)
0.2595 - Applying GPGPU to Recurrent Neural Network Language Model based Fast Network Search in the Real-Time LVCSR (http://arxiv.org/abs/2007.11794v1)
0.2560 - DQI: A Guide to Benchmark Evaluation (http://arxiv.org

In [ ]:
import arxiv
from sentence_transformers import SentenceTransformer, util
import os

# ----------------------------------------------------
# 0. Create download directory
# ----------------------------------------------------
download_dir = "downloads"
os.makedirs(download_dir, exist_ok=True)

# ----------------------------------------------------
# 1. Load an embedding model
# ----------------------------------------------------
model = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------------------------------
# 2. Get abstract of a given paper
# ----------------------------------------------------
paper_id = "2511.20636"  # example
search = arxiv.Search(id_list=[paper_id])
paper = list(search.results())[0]
target_abstract = paper.summary

# ----------------------------------------------------
# 3. Encode the target paper
# ----------------------------------------------------
target_emb = model.encode(target_abstract, convert_to_tensor=True)

# ----------------------------------------------------
# 4. Search for candidate papers in the same category
# ----------------------------------------------------
search_query = "cat:cs.CL"  # choose your category
results = arxiv.Search(query=search_query, max_results=200)

candidate_papers = []
candidate_abstracts = []

for r in results.results():
    if r.entry_id.endswith(paper_id):
        continue
    candidate_papers.append(r)
    candidate_abstracts.append(r.summary)

# ----------------------------------------------------
# 5. Encode candidate abstracts
# ----------------------------------------------------
candidate_embs = model.encode(candidate_abstracts, convert_to_tensor=True)

# ----------------------------------------------------
# 6. Compute similarities
# ----------------------------------------------------
cos_scores = util.cos_sim(target_emb, candidate_embs)[0]

# ----------------------------------------------------
# 7. Get top-k similar papers
# ----------------------------------------------------
top_k = 10
top_results = cos_scores.topk(top_k)

print("Top similar papers:\n")

for score, idx in zip(top_results.values, top_results.indices):
    p = candidate_papers[idx]
    print(f"{score:.4f} - {p.title} ({p.entry_id})")

    # ----------------------------------------------------
    # 8. Download the PDF
    # ----------------------------------------------------
    pdf_path = os.path.join(download_dir, f"{p.get_short_id()}.pdf")
    print(f"  Downloading to: {pdf_path}")

    p.download_pdf(dirpath=download_dir, filename=f"{p.get_short_id()}.pdf")


/tmp/ipykernel_12882/3682888443.py:21: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  paper = list(search.results())[0]
/tmp/ipykernel_12882/3682888443.py:38: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for r in results.results():


Top similar papers:

0.4052 - ShapeGlot: Learning Language for Shape Differentiation (http://arxiv.org/abs/1905.02925v1)
0.3416 - Intelligent requirements engineering from natural language and their chaining toward CAD models (http://arxiv.org/abs/2007.07825v1)
0.3105 - Listening while Speaking and Visualizing: Improving ASR through Multimodal Chain (http://arxiv.org/abs/1906.00579v3)
0.2871 - Hierarchical Decision Making by Generating and Following Natural Language Instructions (http://arxiv.org/abs/1906.00744v5)
0.2846 - A Study on Effects of Implicit and Explicit Language Model Information for DBLSTM-CTC Based Handwriting Recognition (http://arxiv.org/abs/2008.01532v1)
0.2744 - The Go Transformer: Natural Language Modeling for Game Play (http://arxiv.org/abs/2007.03500v3)
0.2595 - Applying GPGPU to Recurrent Neural Network Language Model based Fast Network Search in the Real-Time LVCSR (http://arxiv.org/abs/2007.11794v1)
0.2560 - DQI: A Guide to Benchmark Evaluation (http://arxiv.or